<a href="https://colab.research.google.com/github/gferew1/IntroML/blob/main/hw7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

# Check for CUDA availability and set the device
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")



CUDA is available. Using GPU.


In [ ]:
# Define the CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.ReLU()
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool1(self.act1(self.conv1(x)))
        x = self.pool2(self.act2(self.conv2(x)))
        x = x.view(-1, 8 * 8 * 8)
        x = self.act3(self.fc1(x))
        x = self.fc2(x)
        return x



In [ ]:
# Prepare CIFAR-10 dataset
data_path = '../data-unversioned/p1ch6/'
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
])

cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform=transform)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)

# DataLoader
train_loader = DataLoader(cifar10, batch_size=64, shuffle=True)
val_loader = DataLoader(cifar10_val, batch_size=64, shuffle=False)

# Setup the model, optimizer, and loss function
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()


100%|██████████| 170498071/170498071 [00:02<00:00, 64326900.82it/s]


Extracting ../data-unversioned/p1ch6/cifar-10-python.tar.gz to ../data-unversioned/p1ch6/
Files already downloaded and verified


In [ ]:
# Initialize variables for metrics
total_training_time = 0
epoch_losses = []

# Modified training loop to record metrics
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    global total_training_time
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        start_time = time.time()

        for imgs, labels in train_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()

        end_time = time.time()
        total_training_time += end_time - start_time
        epoch_loss = loss_train / len(train_loader)
        epoch_losses.append(epoch_loss)

        if epoch == 1 or epoch % 10 == 0 or epoch == n_epochs:
            print(f'Epoch {epoch}, Training loss {epoch_loss}')


In [ ]:
# Train the model
training_loop(
    n_epochs=300,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    train_loader=train_loader
)

# Modified validation to record accuracy
def validate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.size(0)
            correct += int((predicted == labels).sum())

    accuracy = correct / total
    return accuracy

accuracy = validate(model, val_loader)


Epoch 1, Training loss 2.118343261958998
Epoch 10, Training loss 1.125154920627394
Epoch 20, Training loss 0.9513692104298136
Epoch 30, Training loss 0.8682655227153807
Epoch 40, Training loss 0.811758241956801
Epoch 50, Training loss 0.7684587559203053
Epoch 60, Training loss 0.7352390052260035
Epoch 70, Training loss 0.7066352240493535
Epoch 80, Training loss 0.685337789108991
Epoch 90, Training loss 0.6661190847149285
Epoch 100, Training loss 0.6541597452538702
Epoch 110, Training loss 0.638415762797341
Epoch 120, Training loss 0.6307086101578324
Epoch 130, Training loss 0.6205525988965388
Epoch 140, Training loss 0.6125045066599346
Epoch 150, Training loss 0.6056700305789328
Epoch 160, Training loss 0.5984811785885745
Epoch 170, Training loss 0.5906635256831908
Epoch 180, Training loss 0.5853971571035093
Epoch 190, Training loss 0.5827668143050445
Epoch 200, Training loss 0.5761483353574562
Epoch 210, Training loss 0.5710281148514784
Epoch 220, Training loss 0.5656265297814098
Epoc

In [ ]:
# Print or save the metrics
print(f"Total Training Time: {total_training_time:.2f} seconds")
print(f"Training Losses Over Epochs: {epoch_losses}")
print(f"Evaluation Accuracy after 300 epochs: {accuracy}")

# Optionally, write these metrics to a file
with open("training_metrics.txt", "w") as file:
    file.write(f"Total Training Time: {total_training_time:.2f} seconds\n")
    file.write(f"Training Losses Over Epochs: {epoch_losses}\n")
    file.write(f"Evaluation Accuracy after 300 epochs: {accuracy}\n")



Total Training Time: 4890.97 seconds
Training Losses Over Epochs: [2.118343261958998, 1.7179432755236126, 1.508513274552572, 1.400537279103418, 1.3360878798510412, 1.2852853150928722, 1.2360740129447654, 1.1944039476954418, 1.1558052023201038, 1.125154920627394, 1.097100122734104, 1.0740516966261218, 1.053207089833896, 1.0346937468442161, 1.016857335558328, 0.999689489145718, 0.9868245939616962, 0.9738566210050412, 0.9635792382995186, 0.9513692104298136, 0.9423371936811511, 0.9317797913270838, 0.9222343018292771, 0.9113693046752754, 0.9055778487868931, 0.8969689478044924, 0.8908786384193489, 0.8827549496956189, 0.877424971526846, 0.8682655227153807, 0.8611227911527809, 0.8540788391209624, 0.8481679895649785, 0.8429487195542401, 0.8407381045086609, 0.8319511379274871, 0.8274875577453458, 0.819866128094361, 0.8165901345974954, 0.811758241956801, 0.8069056266790156, 0.8003277340356041, 0.7981083213215898, 0.7929411341848276, 0.7875303808228135, 0.7837374993526113, 0.7809744037097067, 0.77

part 1B

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

# Check for CUDA availability and set the device
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")


In [ ]:
# Modified CNN model with an additional convolutional layer
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(32, 16, kernel_size=3, padding=1)  # Additional layer
        self.act3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2)
        # Adjusted fully connected layer
        self.fc1 = nn.Linear(16 * 4 * 4, 32)  # Adjust dimensions accordingly
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool1(self.act1(self.conv1(x)))
        x = self.pool2(self.act2(self.conv2(x)))
        x = self.pool3(self.act3(self.conv3(x)))  # Additional layer
        x = x.view(-1, 16 * 4 * 4)  # Adjust view
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [ ]:
# Prepare CIFAR-10 dataset
data_path = '../data-unversioned/p1ch6/'
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
])


In [ ]:
cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform=transform)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)

# DataLoader
train_loader = DataLoader(cifar10, batch_size=64, shuffle=True)
val_loader = DataLoader(cifar10_val, batch_size=64, shuffle=False)

# Setup the model, optimizer, and loss function
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

# Initialize variables for metrics
total_training_time = 0
epoch_losses = []


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Modified training loop to record metrics
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    global total_training_time
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        start_time = time.time()

        for imgs, labels in train_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()

        end_time = time.time()
        total_training_time += end_time - start_time
        epoch_loss = loss_train / len(train_loader)
        epoch_losses.append(epoch_loss)

        if epoch == 1 or epoch % 10 == 0 or epoch == n_epochs:
            print(f'Epoch {epoch}, Training loss {epoch_loss}')


In [ ]:
# Train the model
import torch.nn.functional as F
training_loop(
    n_epochs=300,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    train_loader=train_loader
)

# Modified validation to record accuracy
def validate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.size(0)
            correct += int((predicted == labels).sum())

    accuracy = correct / total
    return accuracy

accuracy = validate(model, val_loader)


Epoch 1, Training loss 2.2197195213773977
Epoch 10, Training loss 1.1506458853211854
Epoch 20, Training loss 0.8708154684899713
Epoch 30, Training loss 0.7458834461010325
Epoch 40, Training loss 0.6697807903103816
Epoch 50, Training loss 0.6141825779091061
Epoch 60, Training loss 0.573175831752665
Epoch 70, Training loss 0.5383918472110768
Epoch 80, Training loss 0.508937921594171
Epoch 90, Training loss 0.4869399197456782
Epoch 100, Training loss 0.46459664997008754
Epoch 110, Training loss 0.4506819081275969
Epoch 120, Training loss 0.43343013387811763
Epoch 130, Training loss 0.42064590293847387
Epoch 140, Training loss 0.4041229218549436
Epoch 150, Training loss 0.39467144260168685
Epoch 160, Training loss 0.3831133645437563
Epoch 170, Training loss 0.37311083522370403
Epoch 180, Training loss 0.36467875285869666
Epoch 190, Training loss 0.3613382588567026
Epoch 200, Training loss 0.3520490653679499
Epoch 210, Training loss 0.34547489210772697
Epoch 220, Training loss 0.33677049605

In [ ]:
# Print or save the metrics
print(f"Total Training Time: {total_training_time:.2f} seconds")
print(f"Training Losses Over Epochs: {epoch_losses}")
print(f"Evaluation Accuracy after 300 epochs: {accuracy}")

# Optionally, write these metrics to a file
with open("training_metrics.txt", "w") as file:
    file.write(f"Total Training Time: {total_training_time:.2f} seconds\n")
    file.write(f"Training Losses Over Epochs: {epoch_losses}\n")
    file.write(f"Evaluation Accuracy after 300 epochs: {accuracy}\n")


Total Training Time: 5096.12 seconds
Training Losses Over Epochs: [2.2197195213773977, 1.8813542089498867, 1.6279902138063669, 1.5174160621050374, 1.427062135065913, 1.359341266819888, 1.2954430019916476, 1.2406638066482056, 1.1935176609269798, 1.1506458853211854, 1.1100826763436007, 1.0754650628475277, 1.0388733674498165, 1.0105130500195887, 0.9821785830933115, 0.9572325076745904, 0.9351888907230114, 0.9098549047699365, 0.8919203967389548, 0.8708154684899713, 0.8560184089424056, 0.8389412990921293, 0.8244503063085439, 0.8103245753613885, 0.7975292185230938, 0.7884911691669918, 0.775555870912569, 0.7645872495210994, 0.7565385330744716, 0.7458834461010325, 0.7364228954324332, 0.728327452336126, 0.718622481083626, 0.7127696617942332, 0.7025803407211133, 0.6953174155157851, 0.6895055943132972, 0.6823412343821562, 0.6753580051538585, 0.6697807903103816, 0.6639307386353802, 0.6575344444235878, 0.6519887934026816, 0.6473861254770737, 0.641550717603825, 0.6353230989345199, 0.6279636736568588,

Problem 2A

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

# Check for CUDA availability and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [ ]:
# Define the ResNet block
class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out


In [ ]:
# Define the ResNet-10
class ResNet10(nn.Module):
    def __init__(self):
        super(ResNet10, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        # Create 10 ResNet blocks
        self.layers = self._make_layers(ResNetBlock, 64, 10, stride=1)

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, 10)  # CIFAR-10 has 10 classes

    def _make_layers(self, block, out_channels, num_blocks, stride):
        layers = []
        for _ in range(num_blocks):
            layers.append(block(self.in_channels, out_channels, stride))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


In [ ]:
# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)

# Create ResNet-10 model
model = ResNet10().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


100%|██████████| 170498071/170498071 [00:02<00:00, 79907856.79it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# Train the model
start_time = time.time()

for epoch in range(300):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/300], Loss: {running_loss / len(train_loader)}')

end_time = time.time()
total_training_time = end_time - start_time

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test images: {accuracy}%')


Epoch [1/300], Loss: 1.3700734099463734
Epoch [2/300], Loss: 0.9747723994962395
Epoch [3/300], Loss: 0.8167755990991812
Epoch [4/300], Loss: 0.7069621771345358
Epoch [5/300], Loss: 0.612974745080904
Epoch [6/300], Loss: 0.5388997607981153
Epoch [7/300], Loss: 0.4838336400516198
Epoch [8/300], Loss: 0.43513788896448474
Epoch [9/300], Loss: 0.3923194239968839
Epoch [10/300], Loss: 0.36220576128234033
Epoch [11/300], Loss: 0.3274009058161465
Epoch [12/300], Loss: 0.3027380798250208
Epoch [13/300], Loss: 0.27605625282010765
Epoch [14/300], Loss: 0.24903656237418084
Epoch [15/300], Loss: 0.22778127938890091
Epoch [16/300], Loss: 0.2102278946229564
Epoch [17/300], Loss: 0.18896981294426468
Epoch [18/300], Loss: 0.17627362483907538
Epoch [19/300], Loss: 0.15205721049319448
Epoch [20/300], Loss: 0.14200500171164723
Epoch [21/300], Loss: 0.13376207582061858
Epoch [22/300], Loss: 0.11619356114064794
Epoch [23/300], Loss: 0.10968716695542684
Epoch [24/300], Loss: 0.10477202123178699
Epoch [25/300

In [ ]:
# Report the training time, loss, and accuracy
print(f"Total Training Time: {total_training_time:.2f} seconds")
print(f"Final Training Loss: {running_loss / len(train_loader)}")
print(f"Test Accuracy: {accuracy}%")
